In [39]:
# Graph Unsupervised Embedding Methods
# ====================================

import re
import osmnx as ox
import numpy as np
import networkx as nx
import torch
import matplotlib.pyplot as plt
import pickle
import os
import glob
import pandas as pd
from tqdm import tqdm
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.cluster import KMeans
from sklearn.manifold import MDS
from sklearn.metrics import f1_score, silhouette_score

from torch_geometric.data import Data
from torch_geometric.nn import global_mean_pool, GCNConv
import torch.nn as nn

# Make sure we have consistent results
torch.manual_seed(42)
np.random.seed(42)

In [44]:
def get_city_names(graph_codes):
    city_names = {}
    for code in graph_codes:
        try:
            place = ox.geocode_to_gdf(f"{code[1:]}")  # Убираем 'R'
            city_name = place['name'].values[0]
            city_names[code] = city_name
        except Exception as e:
            print(f"Не удалось загрузить город для {code}: {e}")
            city_names[code] = "Unknown"
    return city_names


In [ ]:
# ====================================
# Part 1: Overview of Graph Embedding Methods
# ====================================

"""
Graph embedding methods transform graph structures into vector representations.
This enables machine learning algorithms to work with graph data.

Types of Graph Embedding Algorithms:
1. Node Embedding Algorithms:
   - Traditional: PCA, MDS, Laplacian Eigenmaps
   - Random Walk-based: DeepWalk, Node2Vec, LINE
   - Neural Network-based: GCN, GraphSAGE, GAT
   - Matrix Factorization: GraRep, HOPE
   - Probabilistic: VGAE, Deep Graph Infomax
   - Structural: struc2vec

2. Edge Embedding Algorithms:
   - Operator-based (Hadamard, average, etc.) on node embeddings
   - Explicit edge embedding methods

3. Whole Graph Embedding Algorithms:
   - Graph Kernels: Weisfeiler-Lehman, Graphlet
   - Neural Methods: Graph2Vec, DGCNN, Readout functions in GNNs
"""

In [ ]:
# ====================================
# Part 2: Graph Data Loading and Conversion
# ====================================

def generate_random_graph(n, p):
    """
    Generate a random Erdos-Renyi graph and convert to PyTorch Geometric format.
    
    Args:
        n (int): Number of nodes
        p (float): Probability of edge creation
        
    Returns:
        Data: PyTorch Geometric graph object
    """
    G = nx.erdos_renyi_graph(n, p)
    
    # Convert to PyTorch Geometric format
    pyg_graph = Data(
        x=torch.ones(G.number_of_nodes(), 1),  # Simple node features (all ones)
        edge_index=torch.tensor(list(G.edges)).t().contiguous()  # Edge list in COO format
    )
    
    return pyg_graph

def load_and_convert_graph(pkl_path, threshold=10):
    """
    Load graph data from pickle file and convert to PyTorch Geometric format.
    
    The pickle file is expected to contain a dictionary with 'dgms' key,
    which contains distance and coordinate information.
    
    Args:
        pkl_path (str): Path to the pickle file
        threshold (float): Threshold distance for creating edges
        
    Returns:
        Data: PyTorch Geometric graph object
    """
    # Load data dictionary from pickle
    with open(pkl_path, "rb") as f:
        data_dict = pickle.load(f)
    
    # Extract distance and coordinate arrays
    distances = data_dict['dgms'][0]   # Distance array
    coordinates = data_dict['dgms'][1]  # Coordinate array

    # Create empty NetworkX graph
    G_net = nx.Graph()
    
    # Add nodes with coordinates
    for i, coord in enumerate(coordinates):
        G_net.add_node(i, pos=tuple(coord))
    
    # Add edges based on distances with threshold
    num_nodes = len(distances)
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if distances[i, 1] < threshold and distances[j, 1] < threshold:
                G_net.add_edge(i, j, weight=distances[i, 1])
    
    # Convert to PyTorch Geometric format
    if G_net.number_of_edges() > 0:
        edge_index = torch.tensor(list(G_net.edges)).t().contiguous()
    else:
        # Create empty edge tensor if no edges
        edge_index = torch.empty((2, 0), dtype=torch.long)
    
    pyg_data = Data(
        x=torch.ones(G_net.number_of_nodes(), 1),  # Simple node features (all ones)
        edge_index=edge_index  # Edge list in COO format
    )
    return pyg_data

In [ ]:
# ====================================
# Part 3: Graph Neural Network for Embedding
# ====================================

class GNNEmbedder(nn.Module):
    """
    Graph Neural Network for generating graph embeddings.
    
    This model uses a Graph Convolutional Network (GCN) layer followed
    by a global pooling operation to create a fixed-size embedding vector
    for each graph.
    """
    def __init__(self, hidden_dim=64):
        """
        Initialize the GNN embedding model.
        
        Args:
            hidden_dim (int): Dimension of the hidden/embedding space
        """
        super().__init__()
        # GCN layer: transforms node features (dim=1) to hidden_dim
        self.conv1 = GCNConv(1, hidden_dim)
        
        # Pooling function to aggregate node embeddings into graph embeddings
        # Alternatives: global_add_pool, global_max_pool
        self.pool = global_mean_pool
        
    def forward(self, data):
        """
        Forward pass to generate graph embeddings.
        
        Args:
            data (Data): PyTorch Geometric graph data object
            
        Returns:
            torch.Tensor: Graph embedding vector
        """
        x, edge_index, batch = data.x, data.edge_index, data.batch
        
        # Apply GCN layer with ReLU activation
        x = self.conv1(x, edge_index).relu()
        
        # Pool node embeddings to get graph embedding
        return self.pool(x, batch)

def get_graph_embedding(graph, model):
    """
    Generate embedding for a single graph.
    
    Args:
        graph (Data): PyTorch Geometric graph data object
        model (nn.Module): GNN model for embedding
        
    Returns:
        numpy.ndarray: Graph embedding vector
    """
    # Set batch indices (single graph: all nodes have batch index 0)
    graph.batch = torch.zeros(graph.num_nodes, dtype=torch.long)
    
    # Generate embedding (no gradient needed)
    with torch.no_grad():
        emb = model(graph)
    
    return emb.numpy()


In [ ]:
# ====================================
# Part 4: Graph Data Analysis Pipeline
# ====================================

def load_graph_dataset(data_dir="./output_3", pattern="*/result.pkl", threshold=10):
    """
    Load all graph data from a directory and convert to PyTorch Geometric format.
    
    Args:
        data_dir (str): Directory containing graph data
        pattern (str): Pattern to match pickle files
        threshold (float): Threshold for edge creation
        
    Returns:
        tuple: (graph_names, pyg_graphs) - Lists of graph names and PyG objects
    """
    # Find all pickle files matching the pattern
    pkl_files = glob.glob(os.path.join(data_dir, pattern))
    print(f"Found {len(pkl_files)} files")
    
    # Load graphs and their names
    graph_names = []
    pyg_graphs = []
    
    for p in tqdm(pkl_files, desc="Loading graphs"):
        try:
            graph = load_and_convert_graph(p, threshold=threshold)
            pyg_graphs.append(graph)
            # Extract graph name from directory name
            graph_names.append(os.path.basename(os.path.dirname(p)))
        except Exception as e:
            print(f"Failed to load graph from {p}: {e}")
    
    print(f"Successfully loaded {len(pyg_graphs)} graphs")
    return graph_names, pyg_graphs

def compute_embeddings(graph_names, pyg_graphs, hidden_dim=64):
    """
    Compute embeddings for a list of graphs.
    
    Args:
        graph_names (list): List of graph names
        pyg_graphs (list): List of PyTorch Geometric graph objects
        hidden_dim (int): Dimension of the embedding space
        
    Returns:
        tuple: (embeddings, graph_names_emb, df_emb) - Embedding array, 
               list of graph names, and distance matrix DataFrame
    """
    # Initialize GNN model
    model = GNNEmbedder(hidden_dim=hidden_dim)
    
    # Compute embeddings
    embeddings = []
    graph_names_emb = []
    
    for name, pyg_graph in tqdm(zip(graph_names, pyg_graphs), 
                                desc="Computing embeddings", 
                                total=len(graph_names)):
        try:
            emb = get_graph_embedding(pyg_graph, model)
            if emb.size == 0 or emb.shape[0] == 0:
                print(f"Graph {name} returned empty embedding. Using zero vector.")
                emb = np.zeros((1, hidden_dim))
            embeddings.append(emb)
            graph_names_emb.append(name)
        except Exception as e:
            print(f"Failed to compute embedding for {name}: {e}")
    
    # Stack embeddings into a single array
    embeddings = np.vstack(embeddings)
    print(f"Embedding shape: {embeddings.shape}")
    
    # Compute Euclidean distance matrix
    emb_distance_matrix = euclidean_distances(embeddings)
    df_emb = pd.DataFrame(emb_distance_matrix, 
                         index=graph_names_emb, 
                         columns=graph_names_emb)
    
    return embeddings, graph_names_emb, df_emb

def analyze_graph_statistics(pyg_graphs):
    """
    Analyze basic statistics of the graph dataset.
    
    Args:
        pyg_graphs (list): List of PyTorch Geometric graph objects
        
    Returns:
        dict: Dictionary of statistics
    """
    # Count nodes for each graph
    node_counts = [graph.num_nodes for graph in pyg_graphs]
    
    # Compute statistics
    stats = {
        "mean_nodes": np.mean(node_counts),
        "median_nodes": np.median(node_counts),
        "std_nodes": np.std(node_counts),
        "min_nodes": np.min(node_counts),
        "max_nodes": np.max(node_counts)
    }
    
    # Plot node count histogram
    plt.figure(figsize=(10, 6))
    plt.hist(node_counts, bins=20, edgecolor='black')
    plt.xlabel('Number of nodes')
    plt.ylabel('Number of graphs')
    plt.title('Distribution of graph sizes (node counts)')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    
    return stats

In [42]:
# ====================================
# Part 5: Graph Clustering and Evaluation
# ====================================

def cluster_and_evaluate(embedding_data, persistent_diagram_data, n_clusters=2):
    """
    Cluster graphs using both embedding and persistent diagram data,
    then evaluate the alignment between the two clustering approaches.
    
    Args:
        embedding_data (DataFrame): Distance matrix from graph embeddings
        persistent_diagram_data (DataFrame): Distance matrix from persistent diagrams
        n_clusters (int): Number of clusters to create
        
    Returns:
        dict: Dictionary with clustering results and evaluation metrics
    """
    # Find common graph names between the two datasets
    common_names = sorted(set(embedding_data.index).intersection(
                          set(persistent_diagram_data.index)))
    print(f"Found {len(common_names)} common graphs")
    
    # Reindex both matrices to have the same order
    df_emb_ordered = embedding_data.loc[common_names, common_names]
    df_pd_ordered = persistent_diagram_data.loc[common_names, common_names]
    
    # Use MDS to convert distance matrices to 2D points for visualization
    mds = MDS(n_components=2, dissimilarity='precomputed', random_state=42)
    embedding_2d = mds.fit_transform(df_emb_ordered)
    pd_2d = mds.fit_transform(df_pd_ordered)
    
    # Cluster using KMeans
    kmeans_emb = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans_pd = KMeans(n_clusters=n_clusters, random_state=42)
    
    labels_emb = kmeans_emb.fit_predict(embedding_2d)
    labels_pd = kmeans_pd.fit_predict(pd_2d)
    
    # Evaluate clustering quality with silhouette score
    sil_emb = silhouette_score(embedding_2d, labels_emb)
    sil_pd = silhouette_score(pd_2d, labels_pd)
    
    # Evaluate agreement between clusterings with F1 score
    # Note: This assumes persistent diagram clustering is ground truth
    f1 = f1_score(labels_pd, labels_emb, average='weighted')
    
    # Store results
    results = {
        "common_graphs": common_names,
        "embedding_2d": embedding_2d,
        "pd_2d": pd_2d,
        "labels_emb": labels_emb,
        "labels_pd": labels_pd,
        "silhouette_emb": sil_emb,
        "silhouette_pd": sil_pd,
        "f1_score": f1
    }
    
    return results

def find_optimal_clusters(embedding_2d, pd_2d, max_clusters=5):
    """
    Find optimal number of clusters using silhouette score.
    
    Args:
        embedding_2d (numpy.ndarray): 2D embedding points
        pd_2d (numpy.ndarray): 2D persistent diagram points
        max_clusters (int): Maximum number of clusters to try
        
    Returns:
        tuple: (optimal_k_emb, optimal_k_pd, scores_emb, scores_pd)
    """
    scores_emb = {}
    scores_pd = {}
    
    for k in range(2, max_clusters + 1):
        # Cluster embeddings
        kmeans_emb = KMeans(n_clusters=k, random_state=42)
        labels_emb = kmeans_emb.fit_predict(embedding_2d)
        score_emb = silhouette_score(embedding_2d, labels_emb)
        scores_emb[k] = score_emb
        
        # Cluster persistent diagrams
        kmeans_pd = KMeans(n_clusters=k, random_state=42)
        labels_pd = kmeans_pd.fit_predict(pd_2d)
        score_pd = silhouette_score(pd_2d, labels_pd)
        scores_pd[k] = score_pd
    
    # Find optimal k (highest silhouette score)
    optimal_k_emb = max(scores_emb, key=scores_emb.get)
    optimal_k_pd = max(scores_pd, key=scores_pd.get)
    
    return optimal_k_emb, optimal_k_pd, scores_emb, scores_pd

def visualize_clustering(embedding_2d, pd_2d, labels_emb, labels_pd, common_names, city_names):
    fig, axes = plt.subplots(1, 2, figsize=(16, 7))

    scatter1 = axes[0].scatter(embedding_2d[:, 0], embedding_2d[:, 1], 
                              c=labels_emb, cmap='viridis', s=50, alpha=0.8)
    axes[0].set_title('Graph Embedding Clusters')
    axes[0].set_xlabel('Dimension 1')
    axes[0].set_ylabel('Dimension 2')
    axes[0].grid(alpha=0.3)

    scatter2 = axes[1].scatter(pd_2d[:, 0], pd_2d[:, 1], 
                              c=labels_pd, cmap='viridis', s=50, alpha=0.8)
    axes[1].set_title('Persistent Diagram Clusters')
    axes[1].set_xlabel('Dimension 1')
    axes[1].set_ylabel('Dimension 2')
    axes[1].grid(alpha=0.3)

    # Добавляем подписи городов
    for i, city in enumerate(city_names):
        axes[0].annotate(city, (embedding_2d[i, 0], embedding_2d[i, 1]), fontsize=9, alpha=0.75)
        axes[1].annotate(city, (pd_2d[i, 0], pd_2d[i, 1]), fontsize=9, alpha=0.75)

    legend1 = axes[0].legend(*scatter1.legend_elements(), title="Clusters")
    axes[0].add_artist(legend1)

    legend2 = axes[1].legend(*scatter2.legend_elements(), title="Clusters")
    axes[1].add_artist(legend2)

    plt.tight_layout()
    plt.show()


In [ ]:
# ====================================
# Part 6: Hybrid Classification Model
# ====================================

class HybridClassifier(nn.Module):
    """
    Hybrid classifier combining graph embeddings with topological features.
    
    This model can be used for supervised classification of graphs
    based on the combination of GNN embeddings and topological features.
    """
    def __init__(self, input_dim, hidden_dim, num_classes):
        """
        Initialize the hybrid classifier.
        
        Args:
            input_dim (int): Dimension of input features (embedding + topological)
            hidden_dim (int): Dimension of hidden layer
            num_classes (int): Number of output classes
        """
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        """
        Forward pass through the classifier.
        
        Args:
            x (torch.Tensor): Input features
            
        Returns:
            torch.Tensor: Classification logits
        """
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

def compute_network_measures(G):
    """
    Compute basic network measures for a graph.
    
    Args:
        G (networkx.Graph): NetworkX graph
        
    Returns:
        numpy.ndarray: Array of network measures
    """
    measures = {
        "avg_degree": np.mean(list(dict(G.degree()).values())),
        "clustering_coeff": nx.average_clustering(G) if G.number_of_edges() > 0 else 0,
        "diameter": nx.diameter(G) if nx.is_connected(G) and G.number_of_edges() > 0 else 0,
        "density": nx.density(G),
        "num_nodes": G.number_of_nodes(),
        "num_edges": G.number_of_edges()
    }
    return np.array(list(measures.values()))

In [ ]:
# ====================================
# Part 7: Main Execution Pipeline
# ====================================

def main():
    """
    Main execution function for the graph embedding and analysis pipeline.
    """
    print("Starting graph embedding and analysis pipeline...")
    
    # 1. Load graph data
    print("\n1. Loading graph data...")
    graph_names, pyg_graphs = load_graph_dataset(threshold=10)
    
    # Извлекаем коды R[number]
    graph_codes = [re.search(r'R\d+', name).group() for name in graph_names]
    city_names_dict = get_city_names(graph_codes)
    city_names = [city_names_dict[code] for code in graph_codes]


    
    # 2. Analyze graph statistics
    print("\n2. Analyzing graph statistics...")
    stats = analyze_graph_statistics(pyg_graphs)
    print(f"Graph statistics:\n{stats}")
    
    # 3. Compute graph embeddings
    print("\n3. Computing graph embeddings...")
    embeddings, graph_names_emb, df_emb = compute_embeddings(
        graph_names, pyg_graphs, hidden_dim=64)
    
    # 4. Load persistent diagram distances (Assuming this file exists)
    print("\n4. Loading persistent diagram distances...")
    try:
        # with open('output_3/bottleneck_distances.pkl', 'rb') as f:
            # pd_distance_matrix = pickle.load(f)
        pd_distance_matrix = pd.read_csv('output_3/bottleneck_matrix.csv', index_col=0)
        print("Successfully loaded persistent diagram distances")
    except FileNotFoundError:
        print("Warning: bottleneck_distances_matrix.pkl not found.")

    
    # 5. Cluster and evaluate
    print("\n5. Clustering and evaluating...")
    results = cluster_and_evaluate(df_emb, pd_distance_matrix, n_clusters=2)
    
    # 6. Find optimal number of clusters
    print("\n6. Finding optimal number of clusters...")
    opt_k_emb, opt_k_pd, scores_emb, scores_pd = find_optimal_clusters(
        results["embedding_2d"], results["pd_2d"])
    print(f"Optimal clusters (embedding): {opt_k_emb}")
    print(f"Optimal clusters (persistent diagrams): {opt_k_pd}")
    print(f"Silhouette scores (embedding): {scores_emb}")
    print(f"Silhouette scores (persistent diagrams): {scores_pd}")
    
    # 7. Visualize clusters
    print("\n7. Visualizing clusters...")
    visualize_clustering(
        results["embedding_2d"], 
        results["pd_2d"], 
        results["labels_emb"], 
        results["labels_pd"], 
        results["common_graphs"], 
        city_names
    )

    
    print("\nPipeline completed successfully!")

if __name__ == "__main__":
    main()